In [1]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import cartopy.crs as ccrs
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

import cartopy.io.shapereader as shpreader
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import matplotlib.patches as mpatches
import geopandas as gpd

import psycopg2
from sqlalchemy import create_engine

https://www.census.gov/topics/public-sector/voting/data/tables.2010.List_1863097513.html#list-tab-List_1863097513

In [2]:
# read in the Excel file and specify the sheet name
df_votes_2012 = pd.read_excel('Election_Results/vote2012.xls', sheet_name='Table 4a', header=[4])

# display the resulting dataframe
df_votes_2012

Unnamed: 0  Unnamed: 1  Unnamed: 2   
0                                       UNITED STATES    235248.0    215081.0  \
1                                             ALABAMA      3594.0      3479.0   
2                                              ALASKA       516.0       495.0   
3                                             ARIZONA      4863.0      4314.0   
4                                            ARKANSAS      2198.0      2109.0   
5                                          CALIFORNIA     28357.0     23419.0   
6                                            COLORADO      3817.0      3544.0   
7                                         CONNECTICUT      2726.0      2499.0   
8                                            DELAWARE       693.0       641.0   
9                                DISTRICT OF COLUMBIA       517.0       461.0   
10                                            FLORIDA     15034.0     13326.0   
11                                            GEORGIA      7179.0      6738.0   
12                                             HAWAII      1013.0       930.0   
13                                              IDAHO      1129.0      1064.0   
14                                           ILLINOIS      9651.0      8831.0   
15                                            INDIANA      4852.0      4724.0   
16                                               IOWA      2320.0      2232.0   
17                                             KANSAS      2120.0      1973.0   
18                                           KENTUCKY      3291.0      3194.0   
19                                          LOUISIANA      3321.0      3239.0   
20                                              MAINE      1042.0      1020.0   
21                                           MARYLAND      4449.0      4007.0   
22                                      MASSACHUSETTS      5170.0      4774.0   
23                                           MICHIGAN      7496.0      7228.0   
24                                          MINNESOTA      4055.0      3903.0   
25                                        MISSISSIPPI      2166.0      2130.0   
26                                           MISSOURI      4521.0      4409.0   
27                                            MONTANA       768.0       754.0   
28                                           NEBRASKA      1371.0      1296.0   
29                                             NEVADA      2039.0      1808.0   
30                                      NEW HAMPSHIRE      1028.0       991.0   
31                                         NEW JERSEY      6730.0      5929.0   
32                                         NEW MEXICO      1553.0      1426.0   
33                                           NEW YORK     15066.0     13082.0   
34                                     NORTH CAROLINA      7265.0      6712.0   
35                                       NORTH DAKOTA       528.0       514.0   
36                                               OHIO      8750.0      8550.0   
37                                           OKLAHOMA      2808.0      2733.0   
38                                             OREGON      2998.0      2806.0   
39                                       PENNSYLVANIA      9847.0      9452.0   
40                                       RHODE ISLAND       817.0       751.0   
41                                     SOUTH CAROLINA      3516.0      3380.0   
42                                       SOUTH DAKOTA       616.0       607.0   
43                                          TENNESSEE      4849.0      4678.0   
44                                              TEXAS     18642.0     16062.0   
45                                               UTAH      1917.0      1793.0   
46                                            VERMONT       496.0       487.0   
47                                           VIRGINIA      6094.0      5645.0   
48                                         WASHINGTON      5230.0      483

In [4]:
# select the columns wanted and rename them
df_selection = df_votes_2012[['Unnamed: 0', 'Unnamed: 1', 'Total registered', 'Total voted']]
df_votes = df_selection.rename(columns={
    'Unnamed: 0': 'State',
    'Unnamed: 1': 'Population',
    'Total registered': 'Registered',
    'Total voted': 'Voted'
})
# add year and office columns
df_votes['Year'] = 2012

# Arrange columns: Move the Year and Office columns to the front
new_cols = ['Year', 'State', 'Population', 'Registered', 'Voted']

# df_votes = df_votes.iloc[1:]  # remove the first row
df_votes = df_votes.iloc[:-3] # removes last 7 rows of text

# Save the first row as a separate DataFrame
first_row = df_votes.iloc[[0]].copy()

# Remove the first row from the original DataFrame
df_votes = df_votes.iloc[1:].copy()

# Append the first row to the end of the DataFrame
df_votes = pd.concat([df_votes, first_row])

df_votes = df_votes.reset_index(drop=True)  # reset the index
df_votes = df_votes.reindex(columns=new_cols) # set year column in front

df_votes
# values are in the thousands

Year                 State  Population  Registered     Voted
0   2012               ALABAMA      3594.0      2556.0    2154.0
1   2012                ALASKA       516.0       361.0     289.0
2   2012               ARIZONA      4863.0      2812.0    2412.0
3   2012              ARKANSAS      2198.0      1376.0    1124.0
4   2012            CALIFORNIA     28357.0     15356.0   13462.0
5   2012              COLORADO      3817.0      2635.0    2495.0
6   2012           CONNECTICUT      2726.0      1760.0    1568.0
7   2012              DELAWARE       693.0       470.0     431.0
8   2012  DISTRICT OF COLUMBIA       517.0       385.0     350.0
9   2012               FLORIDA     15034.0      9102.0    8107.0
10  2012               GEORGIA      7179.0      4767.0    4168.0
11  2012                HAWAII      1013.0       547.0     480.0
12  2012                 IDAHO      1129.0       745.0     679.0
13  2012              ILLINOIS      9651.0      6425.0    5428.0
14  2012               INDIANA      4852.0      3270.0    2801.0
15  2012                  IOWA      2320.0      1745.0    1548.0
16  2012                KANSAS      2120.0      1467.0    1249.0
17  2012              KENTUCKY      3291.0      2303.0    1895.0
18  2012             LOUISIANA      3321.0      2498.0    2148.0
19  2012                 MAINE      1042.0       787.0     700.0
20  2012              MARYLAND      4449.0      2888.0    2609.0
21  2012         MASSACHUSETTS      5170.0      3759.0    3382.0
22  2012              MICHIGAN      7496.0      5620.0    4832.0
23  2012             MINNESOTA      4055.0      3085.0    2859.0
24  2012           MISSISSIPPI      2166.0      1794.0    1588.0
25  2012              MISSOURI      4521.0      3384.0    2818.0
26  2012               MONTANA       768.0       553.0     495.0
27  2012              NEBRASKA      1371.0       901.0     798.0
28  2012                NEVADA      2039.0      1176.0    1048.0
29  2012         NEW HAMPSHIRE      1028.0       752.0     688.0
30  2012            NEW JERSEY      6730.0      4326.0    3670.0
31  2012            NEW MEXICO      1553.0       978.0     878.0
32  2012              NEW YORK     15066.0      8887.0    7675.0
33  2012        NORTH CAROLINA      7265.0      5295.0    4624.0
34  2012          NORTH DAKOTA       528.0       383.0     328.0
35  2012                  OHIO      8750.0      6076.0    5395.0
36  2012              OKLAHOMA      2808.0      1806.0    1431.0
37  2012                OREGON      2998.0      2086.0    1897.0
38  2012          PENNSYLVANIA      9847.0      6795.0    5824.0
39  2012          RHODE ISLAND       817.0       552.0     469.0
40  2012        SOUTH CAROLINA      3516.0      2479.0    2187.0
41  2012          SOUTH DAKOTA       616.0       454.0     370.0
42  2012             TENNESSEE      4849.0      3210.0    2606.0
43  2012                 TEXAS     18642.0     10749.0    8643.0
44  2012                  UTAH      1917.0      1138.0    1022.0
45  2012               VERMONT       496.0       357.0     308.0
46  2012              VIRGINIA      6094.0      4210.0    3778.0
47  2012            WASHINGTON      5230.0      3533.0    3172.0
48  2012         WEST VIRGINIA      1452.0       982.0     690.0
49  2012             WISCONSIN      4352.0      3318.0    3127.0
50  2012               WYOMING       427.0       268.0     247.0
51  2012         UNITED STATES    235248.0    153157.0  132948.0

In [5]:
state_abbr = {
    'ALABAMA': 'AL',
    'ALASKA': 'AK',
    'ARIZONA': 'AZ',
    'ARKANSAS': 'AR',
    'CALIFORNIA': 'CA',
    'COLORADO': 'CO',
    'CONNECTICUT': 'CT',
    'DELAWARE': 'DE',
    'DISTRICT OF COLUMBIA': 'DC',
    'FLORIDA': 'FL',
    'GEORGIA': 'GA',
    'HAWAII': 'HI',
    'IDAHO': 'ID',
    'ILLINOIS': 'IL',
    'INDIANA': 'IN',
    'IOWA': 'IA',
    'KANSAS': 'KS',
    'KENTUCKY': 'KY',
    'LOUISIANA': 'LA',
    'MAINE': 'ME',
    'MARYLAND': 'MD',
    'MASSACHUSETTS': 'MA',
    'MICHIGAN': 'MI',
    'MINNESOTA': 'MN',
    'MISSISSIPPI': 'MS',
    'MISSOURI': 'MO',
    'MONTANA': 'MT',
    'NEBRASKA': 'NE',
    'NEVADA': 'NV',
    'NEW HAMPSHIRE': 'NH',
    'NEW JERSEY': 'NJ',
    'NEW MEXICO': 'NM',
    'NEW YORK': 'NY',
    'NORTH CAROLINA': 'NC',
    'NORTH DAKOTA': 'ND',
    'OHIO': 'OH',
    'OKLAHOMA': 'OK',
    'OREGON': 'OR',
    'PENNSYLVANIA': 'PA',
    'RHODE ISLAND': 'RI',
    'SOUTH CAROLINA': 'SC',
    'SOUTH DAKOTA': 'SD',
    'TENNESSEE': 'TN',
    'TEXAS': 'TX',
    'UTAH': 'UT',
    'VERMONT': 'VT',
    'VIRGINIA': 'VA',
    'WASHINGTON': 'WA',
    'WEST VIRGINIA': 'WV',
    'WISCONSIN': 'WI',
    'WYOMING': 'WY',
    'UNITED STATES': 'US'
}

In [6]:
df_votes['State'] = df_votes['State'].map(state_abbr)
df_votes

Year State  Population  Registered     Voted
0   2012    AL      3594.0      2556.0    2154.0
1   2012    AK       516.0       361.0     289.0
2   2012    AZ      4863.0      2812.0    2412.0
3   2012    AR      2198.0      1376.0    1124.0
4   2012    CA     28357.0     15356.0   13462.0
5   2012    CO      3817.0      2635.0    2495.0
6   2012    CT      2726.0      1760.0    1568.0
7   2012    DE       693.0       470.0     431.0
8   2012    DC       517.0       385.0     350.0
9   2012    FL     15034.0      9102.0    8107.0
10  2012    GA      7179.0      4767.0    4168.0
11  2012    HI      1013.0       547.0     480.0
12  2012    ID      1129.0       745.0     679.0
13  2012    IL      9651.0      6425.0    5428.0
14  2012    IN      4852.0      3270.0    2801.0
15  2012    IA      2320.0      1745.0    1548.0
16  2012    KS      2120.0      1467.0    1249.0
17  2012    KY      3291.0      2303.0    1895.0
18  2012    LA      3321.0      2498.0    2148.0
19  2012    ME      1042.0       787.0     700.0
20  2012    MD      4449.0      2888.0    2609.0
21  2012    MA      5170.0      3759.0    3382.0
22  2012    MI      7496.0      5620.0    4832.0
23  2012    MN      4055.0      3085.0    2859.0
24  2012    MS      2166.0      1794.0    1588.0
25  2012    MO      4521.0      3384.0    2818.0
26  2012    MT       768.0       553.0     495.0
27  2012    NE      1371.0       901.0     798.0
28  2012    NV      2039.0      1176.0    1048.0
29  2012    NH      1028.0       752.0     688.0
30  2012    NJ      6730.0      4326.0    3670.0
31  2012    NM      1553.0       978.0     878.0
32  2012    NY     15066.0      8887.0    7675.0
33  2012    NC      7265.0      5295.0    4624.0
34  2012    ND       528.0       383.0     328.0
35  2012    OH      8750.0      6076.0    5395.0
36  2012    OK      2808.0      1806.0    1431.0
37  2012    OR      2998.0      2086.0    1897.0
38  2012    PA      9847.0      6795.0    5824.0
39  2012    RI       817.0       552.0     469.0
40  2012    SC      3516.0      2479.0    2187.0
41  2012    SD       616.0       454.0     370.0
42  2012    TN      4849.0      3210.0    2606.0
43  2012    TX     18642.0     10749.0    8643.0
44  2012    UT      1917.0      1138.0    1022.0
45  2012    VT       496.0       357.0     308.0
46  2012    VA      6094.0      4210.0    3778.0
47  2012    WA      5230.0      3533.0    3172.0
48  2012    WV      1452.0       982.0     690.0
49  2012    WI      4352.0      3318.0    3127.0
50  2012    WY       427.0       268.0     247.0
51  2012    US    235248.0    153157.0  132948.0

In [7]:
# Calculate the normalized voter count
df_votes['Norm_Voter_Reg'] = df_votes['Voted'] / df_votes['Registered']
df_votes['Norm_Voter_Pop'] = df_votes['Voted'] / df_votes['Population']
df_votes

Year State  Population  Registered     Voted  Norm_Voter_Reg   
0   2012    AL      3594.0      2556.0    2154.0        0.842723  \
1   2012    AK       516.0       361.0     289.0        0.800554   
2   2012    AZ      4863.0      2812.0    2412.0        0.857752   
3   2012    AR      2198.0      1376.0    1124.0        0.816860   
4   2012    CA     28357.0     15356.0   13462.0        0.876661   
5   2012    CO      3817.0      2635.0    2495.0        0.946869   
6   2012    CT      2726.0      1760.0    1568.0        0.890909   
7   2012    DE       693.0       470.0     431.0        0.917021   
8   2012    DC       517.0       385.0     350.0        0.909091   
9   2012    FL     15034.0      9102.0    8107.0        0.890683   
10  2012    GA      7179.0      4767.0    4168.0        0.874344   
11  2012    HI      1013.0       547.0     480.0        0.877514   
12  2012    ID      1129.0       745.0     679.0        0.911409   
13  2012    IL      9651.0      6425.0    5428.0        0.844825   
14  2012    IN      4852.0      3270.0    2801.0        0.856575   
15  2012    IA      2320.0      1745.0    1548.0        0.887106   
16  2012    KS      2120.0      1467.0    1249.0        0.851397   
17  2012    KY      3291.0      2303.0    1895.0        0.822840   
18  2012    LA      3321.0      2498.0    2148.0        0.859888   
19  2012    ME      1042.0       787.0     700.0        0.889454   
20  2012    MD      4449.0      2888.0    2609.0        0.903393   
21  2012    MA      5170.0      3759.0    3382.0        0.899707   
22  2012    MI      7496.0      5620.0    4832.0        0.859786   
23  2012    MN      4055.0      3085.0    2859.0        0.926742   
24  2012    MS      2166.0      1794.0    1588.0        0.885173   
25  2012    MO      4521.0      3384.0    2818.0        0.832742   
26  2012    MT       768.0       553.0     495.0        0.895118   
27  2012    NE      1371.0       901.0     798.0        0.885683   
28  2012    NV      2039.0      1176.0    1048.0        0.891156   
29  2012    NH      1028.0       752.0     688.0        0.914894   
30  2012    NJ      6730.0      4326.0    3670.0        0.848359   
31  2012    NM      1553.0       978.0     878.0        0.897751   
32  2012    NY     15066.0      8887.0    7675.0        0.863621   
33  2012    NC      7265.0      5295.0    4624.0        0.873277   
34  2012    ND       528.0       383.0     328.0        0.856397   
35  2012    OH      8750.0      6076.0    5395.0        0.887920   
36  2012    OK      2808.0      1806.0    1431.0        0.792359   
37  2012    OR      2998.0      2086.0    1897.0        0.909396   
38  2012    PA      9847.0      6795.0    5824.0        0.857101   
39  2012    RI       817.0       552.0     469.0        0.849638   
40  2012    SC      3516.0      2479.0    2187.0        0.882211   
41  2012    SD       616.0       454.0     370.0        0.814978   
42  2012    TN      4849.0      3210.0    2606.0        0.811838   
43  2012    TX     18642.0     10749.0    8643.0        0.804075   
44  2012    UT      1917.0      1138.0    1022.0        0.898067   
45  2012    VT       496.0       357.0     308.0        0.862745   
46  2012    VA      6094.0      4210.0    3778.0        0.897387   
47  2012    WA      5230.0      3533.0    3172.0        0.897821   
48  2012    WV      1452.0       982.0     690.0        0.702648   
49  2012    WI      4352.0      3318.0    3127.0        0.942435   
50  2012    WY       427.0       268.0     247.0        0.921642   
51  2012    US    235248.0    153157.0  132948.0        0.868050   

    Norm_Voter_Pop  
0         0.599332  
1         0.560078  
2         0.495990  
3         0.511374  
4         0.474733  
5         0.653655  
6         0.575202  
7         0.621934  
8         0.676983  
9         0.539244  
10        0.580582  
11        0.473840  
12        0.601417  
13        0.562429  
14        0.577288  
15        0.667241  
16        0.589151  
17        0.575813  


# Add to sql db

In [8]:
# import psycopg2
# from sqlalchemy import create_engine

# Replace the values below with your database credentials
DATABASE = "electiondb"
USER = "postgres"
PASSWORD = "YourPassword"
HOST = "localhost"
PORT = "5432"

# Connect to your PostgreSQL database
conn = psycopg2.connect(
    database=DATABASE,
    user=USER,
    password=PASSWORD,
    host=HOST,
    port=PORT
)

# Create a sqlalchemy engine
engine = create_engine(f"postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")

# Upload the data frames to the database using the voters table
df_votes.to_sql("voters", engine, if_exists="append", index=False)
print("Votes table loaded successfully")

# Commit the changes to the database
conn.commit()

# Close the database connection
conn.close()


Votes table loaded successfully


In [9]:
# import psycopg2
# import pandas as pd

# Replace the values below with your database credentials
DATABASE = "electiondb"
USER = "postgres"
PASSWORD = "YourPassword"
HOST = "localhost"
PORT = "5432"

# Connect to the PostgreSQL database
conn = psycopg2.connect(database=DATABASE, user=USER, password=PASSWORD, host=HOST, port=PORT)

# Query the table and store the results in a Pandas dataframe
df = pd.read_sql_query("SELECT * FROM voters", conn)

# Print the dataframe to verify that it contains data
print(df)

# Close the database connection
conn.close()
# gives 1128 rows for 2020

     Year State  Population  Registered   Voted  Norm_Voter_Reg   
0    2020    AL      3769.0        2527    2247        0.889197  \
1    2020    AK       528.0         383     330        0.861619   
2    2020    AZ      5638.0        3878    3649        0.940949   
3    2020    AR      2283.0        1361    1186        0.871418   
4    2020    CA     30342.0       18001   16893        0.938448   
..    ...   ...         ...         ...     ...             ...   
307  2012    WA      5230.0        3533    3172        0.897821   
308  2012    WV      1452.0         982     690        0.702648   
309  2012    WI      4352.0        3318    3127        0.942435   
310  2012    WY       427.0         268     247        0.921642   
311  2012    US    235248.0      153157  132948        0.868050   

     Norm_Voter_Pop  
0          0.596179  
1          0.625000  
2          0.647215  
3          0.519492  
4          0.556753  
..              ...  
307        0.606501  
308        0.475207